This is jupyter notebook for test training runs

In [2]:
import pandas as pd
import numpy as np
from FNN import FNN
from gradient_descent import *
from ActivFunctions import  *
from LossFunctions import *

In [3]:
path_data = "data\wdbc.data"#data from https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic
regGoal = "radius1"

In [4]:
epochs = 1000
learning_rate = 0.001
batch_size = 24
loss_derivative = MeanSquaredErrorDerivative

In [5]:
columnNames = ["ID number","Diagnosis","radius1","texture1","perimeter1","area1","smoothness1","compactness1","concavity1","concave points1","symmetry1","fractal dimension1","radius2","texture2","perimeter2","area2","smoothness2","compactness2","concavity2","concave points2","symmetry2","fractal dimension2","radius3","texture3","perimeter3","area3","smoothness3","compactness3","concavity3","concave points3","symmetry3","fractal dimension3"]
data_raw = pd.read_csv(path_data,names = columnNames)
data_NoId = data_raw.drop(columns="ID number")#no need for id

In [6]:
#
data = data_NoId.drop(columns="Diagnosis")#categorical variable is unneeded
data = data.iloc[:,0:10]#using only mean features
dataInput = data.drop(columns=regGoal).to_numpy()
dataTarget = data.loc[:,regGoal].to_numpy()
#
dataInput = dataInput.T
dataTarget = dataTarget[np.newaxis,:]

In [7]:
#declaring network
net = FNN([9,64, 32, 16, 8, 4, 2, 1],
          [relu,identity],
          method_ini="HeUni"
)

In [8]:
n_samples = dataTarget.size
indices = np.arange(0,n_samples)
indices_test = np.random.permutation(round(n_samples*0.2))
indices_train = list(set(indices) - set(indices_test))

In [9]:
#
dataInput_train = dataInput[:,indices_train]
dataTarget_train = dataTarget[:,indices_train]
#
dataInput_test = dataInput[:,indices_test]
dataTarget_test = dataTarget[:,indices_test]
#
meanDataInput_train = np.mean(dataInput_train, axis=1)
meanDataInput_train = meanDataInput_train[:,np.newaxis]
stdDataInput_train = np.std(dataInput_train, axis=1)
stdDataInput_train = stdDataInput_train[:,np.newaxis]
dataInput_train = (dataInput_train - meanDataInput_train) / stdDataInput_train
dataInput_test =  (dataInput_test - meanDataInput_train) / stdDataInput_train

In [10]:
net_trained = train_minibatch_sgd(net, dataInput_train, dataTarget_train, epochs, learning_rate, batch_size, loss_derivative, grad_clip = 2)

In [11]:
out = net_trained.forward(dataInput_test)
predict = out[-1]
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

677.119496587001


In [12]:
net_trained_adam = train_minibatch_adam(net,dataInput_train,dataTarget_train,epochs,learning_rate,batch_size,loss_derivative,beta1=0.9,beta2=0.999,epsilon=1e-8)

In [13]:
out = net_trained_adam.forward(dataInput_test)
predict = out[-1]
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

677.1529885654093


Class

In [14]:
epochs = 1000
learning_rate = 0.001
batch_size = 24
loss_derivative = SoftmaxCrossEntropyDerivative

In [15]:
dataClass = data_NoId
dataInput= dataClass.iloc[:,1:].to_numpy()#all features without without diagn
dataTarget = dataClass.loc[:,"Diagnosis"].to_numpy()
#
dataInput = dataInput.T
dataTarget = dataTarget[np.newaxis,:]
dataTarget_one_hot = one_hot_encode(dataTarget,2)

In [16]:
n_samples = dataTarget.size
indices = np.arange(0,n_samples)
indices_test = np.random.permutation(round(n_samples*0.2))
indices_train = list(set(indices) - set(indices_test))

In [17]:
#
dataInput_train = dataInput[:,indices_train]
dataTarget_train = dataTarget_one_hot[:,indices_train]
#
dataInput_test = dataInput[:,indices_test]
dataTarget_test = dataTarget_one_hot[:,indices_test]
#
meanDataInput_train = np.mean(dataInput_train, axis=1)
meanDataInput_train = meanDataInput_train[:,np.newaxis]
stdDataInput_train = np.std(dataInput_train, axis=1)
stdDataInput_train = stdDataInput_train[:,np.newaxis]
dataInput_train = (dataInput_train - meanDataInput_train) / stdDataInput_train
dataInput_test =  (dataInput_test - meanDataInput_train) / stdDataInput_train

In [18]:
#declaring network
net = FNN([30,64, 32, 16, 8, 4, 2, 2],
          [relu,softmax],
          method_ini="HeUni"
)

In [19]:
net_trained = train_minibatch_sgd(net, dataInput_train, dataTarget_train, epochs, learning_rate, batch_size, loss_derivative, grad_clip = 2)

In [20]:
out = net_trained.forward(dataInput_test)
predict = out[-1]
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

24.342368388470987


In [21]:
net_trained_adam = train_minibatch_adam(net,dataInput_train,dataTarget_train,epochs,learning_rate,batch_size,loss_derivative,beta1=0.9,beta2=0.999,epsilon=1e-8)

In [22]:
out = net_trained_adam.forward(dataInput_test)
print(predict[-1].shape)
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

(2, 114)
24.342368388470987


In [24]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [26]:
print(X.shape)

(70000, 784)


In [27]:
print(y.shape)

(70000,)


In [121]:
epochs = 40
learning_rate = 0.0001
batch_size = 128
loss_derivative = SoftmaxCrossEntropyDerivative

In [122]:
#
dataInput = X.T
dataTarget = y[np.newaxis,:]
dataTarget_one_hot = one_hot_encode(dataTarget,10)

In [123]:
n_samples = dataTarget.size
indices = np.arange(0,n_samples)
indices_test = np.random.permutation(round(n_samples*0.2))
indices_train = list(set(indices) - set(indices_test))

In [124]:
#
dataInput_train = dataInput[:,indices_train]
dataTarget_train = dataTarget_one_hot[:,indices_train]
#
dataInput_test = dataInput[:,indices_test]
dataTarget_test = dataTarget_one_hot[:,indices_test]
#
#meanDataInput_train = np.mean(dataInput_train, axis=1)
#meanDataInput_train = meanDataInput_train[:,np.newaxis]
#stdDataInput_train = np.std(dataInput_train, axis=1)
##stdDataInput_train = stdDataInput_train[:,np.newaxis]
dataInput_train = dataInput_train/255.0
dataInput_test =  dataInput_test/255.0

In [115]:
print(dataInput_train.shape)

(784, 56000)


In [125]:
net = FNN([784,1024,512, 256, 128, 10],
          [relu,softmax],
          method_ini="HeNor"
)

In [126]:
net_trained = train_minibatch_sgd(net, dataInput_train, dataTarget_train, epochs, learning_rate, batch_size, loss_derivative, grad_clip = 0)

In [118]:
out = net_trained.forward(dataInput_test)
predict = out[-1]
error = SoftmaxCrossEntropy(dataTarget_test,predict[-1])
print(error)
k= predict[-1]
print(k.shape)
print(k[:,0:5])

2.3091621454479223
(10, 14000)
[[0.12669116 0.09573001 0.14546016 0.14978522 0.09510933]
 [0.07894617 0.09893888 0.14341182 0.09775835 0.09021019]
 [0.09521793 0.0979368  0.12230696 0.11181974 0.12249965]
 [0.14249307 0.16712235 0.08506561 0.13919223 0.11482672]
 [0.08080173 0.11919483 0.07410284 0.07226527 0.08748338]
 [0.08785315 0.0699728  0.05286446 0.09864604 0.07745831]
 [0.12431174 0.12257055 0.09958022 0.07642756 0.09829357]
 [0.06680774 0.06327849 0.08860118 0.0685471  0.08490025]
 [0.08207743 0.08128538 0.05842024 0.10300835 0.08925293]
 [0.11479988 0.0839699  0.1301865  0.08255013 0.13996568]]


In [107]:
net_trained_adam = train_minibatch_adam(net,dataInput_train,dataTarget_train,epochs,learning_rate,batch_size,loss_derivative,beta1=0.9,beta2=0.999,epsilon=1e-8)

In [109]:
out = net_trained_adam.forward(dataInput_test)
print(predict[-1].shape)
error = SoftmaxCrossEntropy(dataTarget_test,predict[-1])
print(error)
k= predict[-1]
print(k[:,0:5])

(10, 14000)
2.4303549310846635
[[0.04346948 0.05979415 0.07906086 0.02882728 0.04460259]
 [0.04748226 0.0627268  0.05523996 0.03901915 0.09899909]
 [0.16015862 0.17720191 0.13857681 0.1567641  0.15903237]
 [0.07132732 0.06566709 0.05807053 0.03864808 0.06951202]
 [0.15049229 0.08318396 0.12100297 0.1469613  0.07427329]
 [0.14320183 0.17186758 0.15092095 0.16307082 0.12157561]
 [0.07368843 0.04772793 0.04094485 0.03695232 0.04111113]
 [0.07022958 0.07047665 0.05654916 0.07713692 0.03132258]
 [0.10534967 0.09757791 0.13535319 0.09162635 0.12580534]
 [0.13460053 0.16377602 0.16428072 0.22099369 0.23376599]]
